In [ ]:
import sys
!{sys.executable} -m pip install bsb[mpi,neuron]==3.7.7 NEURON==8.0.0 nrn-glia==0.3.9 nrn-patch==3.0.0b6 dbbs-models==1.5.0rc0 arborize==2.0.0b8 pynb-sim==0.2.0 --quiet

# Cerebellar scaffold with multicompartmental simulations

This use-case demonstrates a simulation of the cerebellar cortex using multicompartmental neuron models. The network is constructed using our scaffold approach network builder. The example is kept small to keep execution times low for this single-core notebook context.

## Notebook structure

1. Code setup
    2. [Network construction](#Network-construction)
    3. [Network simulation](#Network-simulation)
2. Analysis
    1. [Voltage recordings](#Voltage-recordings)
    2. [Spike raster](#Spike-raster)

## Code setup

### Network construction

The `bsb` package can reconstruct networks based on a configuration file. For multicompartmental models it also requires an extra HDF5 file containing processed morphologies. For this notebook we provide `morphologies.hdf5` containing the cerebellar granule cell, Golgi cell, Purkinje cell, stellate cell and basket cell. 

We're going to create a new configuration file based on the template `mouse_cerebellum_cortex.json`:

In [ ]:
!bsb make-config -t mouse_cerebellum_cortex.json network_configuration.json

And compile a network from it (this may take up to 10 minutes):

In [ ]:
!bsb -v 3 compile -x 50 -z 50

#### Full network plot (only cell soma)

<span style="color: darkred">Attention!</span> These are heavy plots and might take long to load or cause the notebook to become unresponsive. <span style="color: red">DO NOT</span> save the notebook with these outputs present! When opening the notebook all plots will be loaded simultaneously and will surely cause the notebook to crash.

In [ ]:
from bsb.core import from_hdf5
from bsb.plotting import plot_network
network = from_hdf5("mouse_cerebellum_cortex.hdf5")
plot_network(network, from_memory=False);

#### Granular layer neurons

<span style="color: darkred">Attention!</span> These are heavy plots and might take long to load or cause the notebook to become unresponsive. <span style="color: red">DO NOT</span> save the notebook with these outputs present! When opening the notebook all plots will be loaded simultaneously and will surely cause the notebook to crash.

In [ ]:
from bsb.core import from_hdf5
from bsb.plotting import plot_detailed_network
import numpy as np
network = from_hdf5("mouse_cerebellum_cortex.hdf5")
ids = np.array([network.get_placement_set("golgi_cell").identifiers[0], *network.get_placement_set("granule_cell").identifiers[0:5]])
plot_detailed_network(network, ids=ids);

#### Molecular layer neurons

<span style="color: darkred">Attention!</span> These are heavy plots and might take long to load or cause the notebook to become unresponsive. <span style="color: red">DO NOT</span> save the notebook with these outputs present! When opening the notebook all plots will be loaded simultaneously and will surely cause the notebook to crash.

In [ ]:
from bsb.core import from_hdf5
from bsb.plotting import plot_detailed_network
import numpy as np
network = from_hdf5("mouse_cerebellum_cortex.hdf5")
ids = np.array([*network.get_placement_set("purkinje_cell").identifiers[:2], *network.get_placement_set("stellate_cell").identifiers[:2]])
plot_detailed_network(network, ids=ids);

### Network simulation

After construction the network file `mouse_cerebellum_cortex.hdf5` now contains all the information required to run the simulation in parallel. On your own machine you can run a simulation using this command (after installing MPI):

    mpiexec -n <number of cores> bsb -v=3 simulate sensory_burst --hdf5=mouse_cerebellum_cortex.hdf5

In this notebook we will be simulating on just 1 core:

In [ ]:
# This patches the incorrect Collab 2 cache path
import os; os.environ["XDG_CACHE_HOME"] = os.path.join(os.getcwd(), ".cache")
!bsb -v=3 simulate sensory_burst --hdf5 mouse_cerebellum_cortex.hdf5

## Analysis

The simulation outputs its recordings into an HDF5 file with 2 categories of `recorders`: `spike_recorder`s and `voltage_recorder`s. In the configuration file spike and voltage recorders have been requested for each cell at the soma.


### Spike raster

Shows the activity of all cells during the entire simulation.

In [ ]:
from bsb.plotting import hdf5_plot_spike_raster
from h5py import File

with File("results_sensory_burst.hdf5", "r") as f:
    # Plot the spike raster
    hdf5_plot_spike_raster(f["/recorders/soma_spikes"], input_region=[6000, 6050])

### Voltage recordings

Shows the membrane potential at the soma.

In [ ]:
from bsb.plotting import hdf5_gather_voltage_traces, plot_traces
from h5py import File

with File("results_sensory_burst.hdf5", "r") as f:
    # Collect traces from cells across multiple recording groups.
    cell_traces = hdf5_gather_voltage_traces(f, "/recorders/", ["soma_voltages"])
    cell_traces.set_legends(["Soma (mV)"])
    cell_traces.set_colors(["Crimson"])
    plot_traces(cell_traces, input_region=[6000, 6050])